### VGG

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch as th
import torch.nn as nn
from dataset import mnist
from util import (
    get_torch_size_string,
    print_model_parameters,
    print_model_layers,
    model_train,
    model_eval,
    model_test
)
np.set_printoptions(precision=3)
th.set_printoptions(precision=3)
%matplotlib inline
%config InlineBackend.figure_format='retina'
print ("PyTorch version:[%s]."%(th.__version__))

PyTorch version:[2.0.1].


### Hyperparameters

In [2]:
device = 'cpu' # cpu / mps
print ("Ready.")

Ready.


### VGG Blocks

In [3]:
def vgg_block(num_convs,out_channels):
    layer_list = []
    for _ in range(num_convs):
        layer_list.append(
            nn.LazyConv2d(
                out_channels = out_channels,
                kernel_size  = 3,
                padding      = 1
            )
        )
        layer_list.append(
            nn.ReLU()
        )
    layer_list.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layer_list)
print ("Ready.")

Ready.


### VGG Nets

In [4]:
class VGG_Class(nn.Module):
    def __init__(
        self,
        arch        = ((1,64), (1,128), (2,256), (2,512), (2,512)),
        num_classes = 1000
    ):
        super(VGG_Class,self).__init__()
        vgg_block_list = []
        for (num_convs,out_channels) in arch:
            vgg_block_list.append(vgg_block(num_convs,out_channels))
        self.net = nn.Sequential(
            *vgg_block_list,
            nn.Flatten(),
            nn.LazyLinear(4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.LazyLinear(4096),
            nn.ReLU(),nn.Dropout(0.5),
            nn.LazyLinear(num_classes)
        )
        self.layer_names = []
        for l_idx,layer in enumerate(self.net):
            layer_name = "%s_%02d"%(type(layer).__name__.lower(),l_idx)
            self.layer_names.append(layer_name)
        
    def forward(self,x):
        """
            Forward propagate
        """
        intermediate_output_list = []
        for layer in self.net:
            x = layer(x)
            intermediate_output_list.append(x)
        final_output = x
        return final_output,intermediate_output_list
    
print ("Ready.")        

Ready.


In [5]:
vgg = VGG_Class(
    arch        = ((2,64),(2,128),(2,256),(2,512),(2,512)),
    num_classes = 1000
)
print ("Ready.")

Ready.


/Users/sj/anaconda3/envs/mujoco/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


### Print model layers

In [6]:
x_torch = th.randn((16,3,224,224)).to(device)
print_model_layers(vgg,x_torch)

batch_size:[16]
[  ] layer:[          input] size:[  16x3x224x224]
[ 0] layer:[  sequential_00] size:[ 16x64x112x112] numel:[  12845056]
[ 1] layer:[  sequential_01] size:[  16x128x56x56] numel:[   6422528]
[ 2] layer:[  sequential_02] size:[  16x256x28x28] numel:[   3211264]
[ 3] layer:[  sequential_03] size:[  16x512x14x14] numel:[   1605632]
[ 4] layer:[  sequential_04] size:[    16x512x7x7] numel:[    401408]
[ 5] layer:[     flatten_05] size:[      16x25088] numel:[    401408]
[ 6] layer:[  lazylinear_06] size:[       16x4096] numel:[     65536]
[ 7] layer:[        relu_07] size:[       16x4096] numel:[     65536]
[ 8] layer:[     dropout_08] size:[       16x4096] numel:[     65536]
[ 9] layer:[  lazylinear_09] size:[       16x4096] numel:[     65536]
[10] layer:[        relu_10] size:[       16x4096] numel:[     65536]
[11] layer:[     dropout_11] size:[       16x4096] numel:[     65536]
[12] layer:[  lazylinear_12] size:[       16x1000] numel:[     16000]


### Prinit model parameters

In [7]:
print_model_parameters(vgg)

[ 0] parameter:[             net.0.0.weight] shape:[    64x3x3x3] numel:[      1728]
[ 1] parameter:[               net.0.0.bias] shape:[          64] numel:[        64]
[ 2] parameter:[             net.0.2.weight] shape:[   64x64x3x3] numel:[     36864]
[ 3] parameter:[               net.0.2.bias] shape:[          64] numel:[        64]
[ 4] parameter:[             net.1.0.weight] shape:[  128x64x3x3] numel:[     73728]
[ 5] parameter:[               net.1.0.bias] shape:[         128] numel:[       128]
[ 6] parameter:[             net.1.2.weight] shape:[ 128x128x3x3] numel:[    147456]
[ 7] parameter:[               net.1.2.bias] shape:[         128] numel:[       128]
[ 8] parameter:[             net.2.0.weight] shape:[ 256x128x3x3] numel:[    294912]
[ 9] parameter:[               net.2.0.bias] shape:[         256] numel:[       256]
[10] parameter:[             net.2.2.weight] shape:[ 256x256x3x3] numel:[    589824]
[11] parameter:[               net.2.2.bias] shape:[         256]